p100 testing small batch

In [7]:
import time
import numpy as np
import torch
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
# from tqdm.notebook import tqdm

In [8]:
from FADAnet.FADAloader import *
from FADAnet.FADAmodule import *

In [9]:
# loss version: loss = loss1 + gamma*loss2/(10*n_support) + theta*loss3/(10*n_support)
n_epoch = 201  # total number of epoch
n_epoch_pt = 0
batch_size = 128
batch_size_test = 512
lr = 0.0007
gamma = 0.06
theta = 0.05
n_support = 7
loss3_margin = 0.7

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
np.random.seed(777)
torch.manual_seed(777)
torch.cuda.manual_seed(777)

In [10]:
train_dataloader=mnist_dataloader(batch_size=batch_size,train=True)
test_dataloader=mnist_dataloader(train=False)
real_test_loader = svhn_dataloader(batch_size = batch_size_test, train=False)

classifier=Classifier()
encoder=Encoder()

classifier.to(device)
encoder.to(device)
loss_fn1 = torch.nn.CrossEntropyLoss()
loss_fn2 = torch.nn.CosineEmbeddingLoss()
loss_fn3 = torch.nn.CosineEmbeddingLoss(margin=loss3_margin)
optimizer=torch.optim.Adadelta(list(encoder.parameters())+list(classifier.parameters()))

X_t, Y_t = create_target_samples(n=n_support)

Using downloaded and verified file: ./data/SVHN/test_32x32.mat


In [11]:
# for epoch in tqdm(range(n_epoch)):
for epoch in range(n_epoch):
    
    for data,labels in train_dataloader:
        data=data.to(device)
        labels=labels.to(device)
        X_t = X_t.to(device)
        Y_t = Y_t.to(device)
        optimizer.zero_grad()
        
        map_s = encoder(data)
        y_pred=classifier(map_s)
        loss1=loss_fn1(y_pred,labels)
        map_t = encoder(X_t)
        
        loss2 = 0
        loss3 = 0
        means_s = []
        # means_t = []
        
        for num in range(10):
#             subset = map_t[Y_t == num]
#             means_t.append(torch.mean(subset, dim = 0))
            subset = map_s[labels == num]
            if len(subset) <= 1: 
                danger = 1
            means_s.append(torch.mean(subset, dim = 0))
        for ctr in range(10*n_support):
            num = Y_t[ctr]
            dd = torch.stack([ map_t[ctr] - means_s[num] ]*10)
            Cplane = torch.stack( [(means_s[i-1] - means_s[i]) for i in range(10)] )
            loss2 += loss_fn2(-dd, Cplane, torch.FloatTensor([-1]*10).to(device))
            loss2 += loss_fn2(dd, Cplane, torch.FloatTensor([-1]*10).to(device))
            Cplane = torch.stack( [( map_t[ctr] - means_s[i] ) for i in range(10) if i != num] )
            loss3 += loss_fn3(dd[:-1], Cplane, torch.FloatTensor([-1]*9).to(device))
        
        if torch.isnan(loss1) or torch.isnan(loss2) or torch.isnan(loss3):
            continue
        loss = loss1 + gamma*loss2/(10*n_support) + theta*loss3/(10*n_support)
        loss.backward()
        optimizer.step()
        
    if epoch%5 == 0: print("loss1:", loss1.item(), "   loss2:", loss2.item(), "   loss3:", loss3.item())
        
    if epoch%10 == 0:
        acc=0
        for data,labels in test_dataloader:
            data=data.to(device)
            labels=labels.to(device)
            y_test_pred=classifier(encoder(data))
            acc+=(torch.max(y_test_pred,1)[1]==labels).float().mean().item()
        accuracy=round(acc / float(len(test_dataloader)), 3)
        print("On source domain: Epoch %d/%d  accuracy: %.3f "%(epoch+1,n_epoch,accuracy))
    
    if epoch%20 == 0:
        mapset_f = []
        labelset_f = []
        for data, labels in train_dataloader:
            data = data.to(device)
            labels=labels.to(device)
            map_f = encoder(data)
            mapset_f.append(map_f)
            labelset_f.append(labels)
        map_f = torch.cat(mapset_f[:-1])
        label_f = torch.cat(labelset_f[:-1])

        means_f = []
        for num in range(10):
            subset = map_f[label_f == num]
            means_f.append(torch.mean(subset, dim = 0))
#        nume = 0
        deno = 0
        acc = 0  
        for data, labels in real_test_loader:
            data = data.to(device)
            labels = labels.to(device)        
            map_ff = encoder(data)
            distTS = []
            for ii in range(10):
                distTS.append(torch.norm((map_ff - means_f[ii]), dim=1))
            distTS = torch.stack(distTS)
            acc+=torch.sum(torch.argmin(distTS, dim=0)==labels)
            
#             for ctr in range(batch_size_test):
#                 others = []
#                 for j in range(10):
#                     num = j
#                     tmp = map_ff[ctr] - means_f[num]
#                     dd_f = torch.stack([tmp]*9)
#                     tmp = [(means_f[num] - means_f[i]) for i in range(10) if i != num]
#                     Cplane_f = torch.stack(tmp)
#                     loss_f = loss_fn2(-dd_f, Cplane_f, torch.FloatTensor([-1]*9).to(device)) + loss_fn2(dd_f, Cplane_f, torch.FloatTensor([-1]*9).to(device))
#                     others.append(loss_f.item())
#                 # print(min(others))
#                 if np.argmin(others) == labels[ctr]: 
#                     nume+=1
            deno += batch_size_test
            if deno > 5100: break
        print("-------------------------------------------------")
#        print("On target domain: Epoch %d/%d  accuracy:"%(epoch+1,n_epoch), nume / deno)
        print("Another one on TD: Epoch %d/%d  accuracy:"%(epoch+1,n_epoch), acc.item() / deno)
        print("-------------------------------------------------")
        
        

loss1: 1.5125409364700317    loss2: 16.49027442932129    loss3: 0.0
On source domain: Epoch 1/201  accuracy: 0.958 
-------------------------------------------------
Another one on TD: Epoch 1/201  accuracy: 0.2154296875
-------------------------------------------------
loss1: 1.4869647026062012    loss2: 9.60530948638916    loss3: 0.007021208293735981
loss1: 1.4713544845581055    loss2: 8.610066413879395    loss3: 0.01938006654381752
On source domain: Epoch 11/201  accuracy: 0.987 
loss1: 1.4611505270004272    loss2: 8.175106048583984    loss3: 0.0
loss1: 1.4762845039367676    loss2: 7.679748058319092    loss3: 0.0917993113398552
On source domain: Epoch 21/201  accuracy: 0.987 
-------------------------------------------------
Another one on TD: Epoch 21/201  accuracy: 0.1431640625
-------------------------------------------------
loss1: 1.4611505270004272    loss2: 7.795801639556885    loss3: 0.19746613502502441
loss1: 1.4611505270004272    loss2: 7.286019802093506    loss3: 0.004757

In [12]:
for epoch in range(n_epoch):
    
    for data,labels in train_dataloader:
        data=data.to(device)
        labels=labels.to(device)
        X_t = X_t.to(device)
        Y_t = Y_t.to(device)
        optimizer.zero_grad()
        
        map_s = encoder(data)
        y_pred=classifier(map_s)
        loss1=loss_fn1(y_pred,labels)
        map_t = encoder(X_t)
        
        loss2 = 0
        loss3 = 0
        means_s = []
        # means_t = []
        
        for num in range(10):
#             subset = map_t[Y_t == num]
#             means_t.append(torch.mean(subset, dim = 0))
            subset = map_s[labels == num]
            if len(subset) <= 1: 
                danger = 1
            means_s.append(torch.mean(subset, dim = 0))
        for ctr in range(10*n_support):
            num = Y_t[ctr]
            dd = torch.stack([ map_t[ctr] - means_s[num] ]*10)
            Cplane = torch.stack( [(means_s[i-1] - means_s[i]) for i in range(10)] )
            loss2 += loss_fn2(-dd, Cplane, torch.FloatTensor([-1]*10).to(device))
            loss2 += loss_fn2(dd, Cplane, torch.FloatTensor([-1]*10).to(device))
            Cplane = torch.stack( [( map_t[ctr] - means_s[i] ) for i in range(10) if i != num] )
            loss3 += loss_fn3(dd[:-1], Cplane, torch.FloatTensor([-1]*9).to(device))
        
        if torch.isnan(loss1) or torch.isnan(loss2) or torch.isnan(loss3):
            continue
        loss = loss1 + gamma*loss2/(10*n_support) + theta*loss3/(10*n_support)
        loss.backward()
        optimizer.step()
        
    if epoch%5 == 0: print("loss1:", loss1.item(), "   loss2:", loss2.item(), "   loss3:", loss3.item())
        
    if epoch%10 == 0:
        acc=0
        for data,labels in test_dataloader:
            data=data.to(device)
            labels=labels.to(device)
            y_test_pred=classifier(encoder(data))
            acc+=(torch.max(y_test_pred,1)[1]==labels).float().mean().item()
        accuracy=round(acc / float(len(test_dataloader)), 3)
        print("On source domain: Epoch %d/%d  accuracy: %.3f "%(epoch+1,n_epoch,accuracy))
    
    if epoch%20 == 0:
        mapset_f = []
        labelset_f = []
        for data, labels in train_dataloader:
            data = data.to(device)
            labels=labels.to(device)
            map_f = encoder(data)
            mapset_f.append(map_f)
            labelset_f.append(labels)
        map_f = torch.cat(mapset_f[:-1])
        label_f = torch.cat(labelset_f[:-1])

        means_f = []
        for num in range(10):
            subset = map_f[label_f == num]
            means_f.append(torch.mean(subset, dim = 0))
#        nume = 0
        deno = 0
        acc = 0  
        for data, labels in real_test_loader:
            data = data.to(device)
            labels = labels.to(device)        
            map_ff = encoder(data)
            distTS = []
            for ii in range(10):
                distTS.append(torch.norm((map_ff - means_f[ii]), dim=1))
            distTS = torch.stack(distTS)
            acc+=torch.sum(torch.argmin(distTS, dim=0)==labels)
            
#             for ctr in range(batch_size_test):
#                 others = []
#                 for j in range(10):
#                     num = j
#                     tmp = map_ff[ctr] - means_f[num]
#                     dd_f = torch.stack([tmp]*9)
#                     tmp = [(means_f[num] - means_f[i]) for i in range(10) if i != num]
#                     Cplane_f = torch.stack(tmp)
#                     loss_f = loss_fn2(-dd_f, Cplane_f, torch.FloatTensor([-1]*9).to(device)) + loss_fn2(dd_f, Cplane_f, torch.FloatTensor([-1]*9).to(device))
#                     others.append(loss_f.item())
#                 # print(min(others))
#                 if np.argmin(others) == labels[ctr]: 
#                     nume+=1
            deno += batch_size_test
            if deno > 5100: break
        print("-------------------------------------------------")
#        print("On target domain: Epoch %d/%d  accuracy:"%(epoch+1,n_epoch), nume / deno)
        print("Another one on TD: Epoch %d/%d  accuracy:"%(epoch+1,n_epoch), acc.item() / deno)
        print("-------------------------------------------------")

loss1: 1.4681755304336548    loss2: 3.4909820556640625    loss3: 0.09560218453407288
On source domain: Epoch 1/201  accuracy: 0.986 
-------------------------------------------------
Another one on TD: Epoch 1/201  accuracy: 0.46796875
-------------------------------------------------
loss1: 1.4611505270004272    loss2: 3.119586944580078    loss3: 0.03658119589090347
loss1: 1.4611505270004272    loss2: 2.426074981689453    loss3: 0.2116917371749878
On source domain: Epoch 11/201  accuracy: 0.989 
loss1: 1.4715681076049805    loss2: 2.5029056072235107    loss3: 0.31949055194854736
loss1: 1.481950283050537    loss2: 2.9085214138031006    loss3: 0.01444072276353836
On source domain: Epoch 21/201  accuracy: 0.987 
-------------------------------------------------
Another one on TD: Epoch 21/201  accuracy: 0.405859375
-------------------------------------------------
loss1: 1.4611505270004272    loss2: 2.0377252101898193    loss3: 0.01241395901888609
loss1: 1.4611505270004272    loss2: 2.01

KeyboardInterrupt: 

In [13]:
# result check
mapset = []
labelset = []
for data, labels in train_dataloader:
    data=data.to(device)
    fmap = encoder(data).cpu().detach().numpy()
    labels=labels.to(device).cpu().detach().numpy()
    mapset.append(fmap)
    labelset.append(labels)

smap = np.vstack(mapset[:-1])
slabel = np.hstack(labelset[:-1])

means = []
dmeans = []

for num in range(10):
    subset1 = smap[slabel == num]
    means1 = np.mean(subset1, axis=0)
    tmp = subset1 - means1
    dists1 = np.linalg.norm(tmp, axis=1)
    means.append(means1)
    dmeans.append(np.mean(dists1))
print(means[0])

for i in range(10):
    for j in range(i+1):
        print(np.linalg.norm(means[i] - means[j]), end=',')
    print()

[ -2.1745386  -10.011938    20.48194     -8.192516     8.475638
  15.92413     -2.5822139   -0.3221105   23.022108    13.155584
  -2.5788212    2.066813   -20.825943    15.767265    -3.7322679
  11.519816     6.278767   -12.817459     4.203614    18.105707
   6.2423835    8.375549   -17.613485   -20.151804    15.842639
   7.831655    17.621256     2.5739157  -31.877438     4.3291283
   0.7243002    9.685326    -3.6911      15.160153    -9.740432
  21.500132    -6.8985434   -1.0789224  -10.07859    -11.179356
 -18.595795    11.691293    -4.747765    20.810911   -17.71167
 -22.918343    -7.025477     0.54644954  -5.8722315  -10.991843
   9.204396    -1.2842579    4.2277765   -7.5103946    2.9380515
  18.825861    56.05866      2.7505198   22.156355    -7.5402465
   0.27074465  -8.500637    -6.7862105  -18.57417   ]
0.0,
153.1926,0.0,
129.90042,135.44826,0.0,
98.43348,91.81101,115.29391,0.0,
123.75458,107.45866,100.80699,101.54213,0.0,
155.96478,153.17664,145.15434,135.71869,144.40678,0.0

In [14]:
tmapset = []
tlabelset = []
for data, labels in real_test_loader:
    data=data.to(device)
    fmap = encoder(data).cpu().detach().numpy()
    labels=labels.to(device).cpu().detach().numpy()
    tmapset.append(fmap)
    tlabelset.append(labels)

tmap = np.vstack(tmapset[:-1])
tlabel = np.hstack(tlabelset[:-1])

tmeans = []
tdmeans = []
for num in range(10):
    subset1 = tmap[tlabel == num]
    means1 = np.mean(subset1, axis=0)
    tmp = subset1 - means1
    dists1 = np.linalg.norm(tmp, axis=1)
    tmeans.append(means1)
    tdmeans.append(np.mean(dists1))
print(dmeans)

[15.62518, 12.894861, 17.233725, 14.968615, 14.968787, 22.451544, 13.444863, 20.46196, 17.3671, 17.675406]


In [15]:
for num in range(10):
    subset1 = tmap[tlabel == num]
    tsd = []
    for i in range(10):
        tmp = np.linalg.norm((subset1 - means[i]), axis=1)
        tsd.append(np.mean(tmp))
    print(tsd)

[109.98859, 145.80316, 133.12094, 112.94388, 127.58434, 158.94243, 113.93175, 154.0673, 129.75859, 132.459]
[141.3419, 111.68559, 128.03937, 107.128395, 118.45304, 154.68042, 113.00985, 135.93904, 121.55752, 129.95499]
[136.916, 135.47264, 104.95659, 116.27617, 115.841545, 153.81941, 109.78008, 136.67548, 120.065155, 125.03355]
[130.44582, 124.80035, 123.91277, 101.12083, 115.628586, 141.42651, 104.802216, 142.43958, 116.96044, 115.77162]
[124.11284, 123.036026, 117.15329, 102.746735, 99.60242, 148.26875, 96.80089, 143.62062, 115.26248, 113.04205]
[143.78268, 138.88487, 133.3918, 119.83562, 126.583084, 123.85974, 114.255684, 156.35088, 126.716064, 123.576866]
[124.64522, 132.73335, 121.918495, 108.7073, 114.166756, 142.40434, 98.1383, 148.37422, 114.488014, 116.86466]
[162.33711, 146.72014, 136.16527, 132.16742, 146.58186, 174.18448, 138.7016, 123.67335, 148.63362, 151.2084]
[129.11009, 130.04749, 122.73905, 108.84991, 114.781364, 146.04251, 103.91417, 148.26897, 108.80294, 119.76096]


In [16]:
print(tmeans[0])

[ -5.034887    -9.137211     9.684193    -5.1165724    5.3417206
   2.586344     1.9177574    2.8247616    4.3527427    4.5672827
  -4.55892     -0.8631613   -3.876222     8.085945    -1.7921355
  -1.4425997    0.84082437  -3.4799209   -1.1960568    7.2356153
   1.5747296    5.0122123   -8.104763   -11.641367     6.3390903
   5.321138    11.980459    -4.5846653   -5.2079797   -1.5730164
  -3.1261814    3.9036183    3.83916      7.6159678  -12.081245
   6.1791134    0.02005372  -1.3215011   -2.6292129   -5.601384
 -12.930199     1.1867195   -2.2178147    7.2312374   -7.9104977
  -9.497583     5.0815043   -2.0728922   -9.720814    -3.7390308
   4.165329    -0.86799586  -8.006124    -5.2386203    0.31658092
   5.4438944  -14.283243     0.6545393    5.9680767   -4.4727
   6.6153975   -8.814387    -6.071163    -4.3301086 ]
